## Notebook to calculate all important metrics for the networks and save them

In [38]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pypsa
import warnings

warnings.filterwarnings("ignore")


# imported own functions
from utils import market_values, capacity, generation
from utils import generation_storage_units, consumption_storage_units, capacity_storage_units, capacity_storage_units_con, market_values_storage_units, market_values_storage_units_con
from utils import generation_links, generation_links_bus, capacity_links, capacity_links_bus, market_values_links, market_values_links_bus

# imported own definitions
from utils import resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal

# general variables
onshore_regions = gpd.read_file("../data/external/regions_onshore_elec_s_181.geojson")
offshore_regions = gpd.read_file("../data/external/regions_offshore_elec_s_181.geojson")
onshore_regions = onshore_regions.set_index('name')
offshore_regions = offshore_regions.set_index('name')

# Regions
onshore_regions['coords'] = onshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
onshore_regions['coords'] = [coords[0] for coords in onshore_regions['coords']]
onshore_regions["name"] = onshore_regions.index
offshore_regions['coords'] = offshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
offshore_regions['coords'] = [coords[0] for coords in offshore_regions['coords']]
offshore_regions["name"] = offshore_regions.index

# carrier lists
from utils import c_el_gen_s, c_el_con_s

# Notebook Definitions
c1_groups = [resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal]
c1_groups_name = ["resistive heater", "gas boiler", "heat pump", "water tanks charger", "water tanks discharger",
                  "solar thermal"]

In [39]:
# Network imports
stst = pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")
exp = pypsa.Network("../data/raw/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc")

Importing PyPSA from older version of PyPSA than current version.
Please read the release notes at https://pypsa.readthedocs.io/en/latest/release_notes.html
carefully to prepare your network for import.
Currently used PyPSA version [0, 20, 1], imported network file PyPSA version [0, 18, 1].

INFO:pypsa.io:Imported network elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
Importing PyPSA from older version of PyPSA than current version.
Please read the release notes at https://pypsa.readthedocs.io/en/latest/release_notes.html
carefully to prepare your network for import.
Currently used PyPSA version [0, 20, 1], imported network file PyPSA version [0, 18, 1].

INFO:pypsa.io:Imported network elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


## Spatial dfs

### General

In [85]:
# calc market values, generation, lmps, capacity factors for generators, links and storage units
for n in [stst, exp]:
    df_regions_onshore = onshore_regions.copy()
    df_regions_offshore = offshore_regions.copy()

    # function for carriers in n.generators.carrier.unique() # 13 / 13
    for carrier in n.generators.carrier.unique():
        df_regions_onshore[f"{carrier}_mv"] = market_values(n, carrier)
        df_regions_offshore[f"{carrier}_mv"] = market_values(n, carrier)
        df_regions_onshore[f"{carrier}_gen"] = generation(n, carrier) / 1000 * 3
        df_regions_offshore[f"{carrier}_gen"] = generation(n, carrier) / 1000 * 3
        df_regions_onshore[f"{carrier}_cap"] = capacity(n, carrier) / 1000
        df_regions_offshore[f"{carrier}_cap"] = capacity(n, carrier) / 1000
        # lmps
        # capacity factors This calculation is correct? as capacity is multiplied by 2920 is the same as multiplying the generation by 3 and then dividing it by the capacity times 8760 (as cap is in MW?)
        df_regions_onshore[f"{carrier}_cf"] = generation(n, carrier) / (capacity(n, carrier) * 2920)
        df_regions_offshore[f"{carrier}_cf"] = generation(n, carrier) / (capacity(n, carrier) * 2920)

    # function for carriers in n.links.carrier.unique() # 53 / 55
    for carrier in n.links.carrier.unique():
        df_regions_onshore[f"{carrier}_mv"] = market_values_links(n, carrier)
        df_regions_onshore[f"{carrier}_gen"] = generation_links(n, carrier) / 1000 * 3
        df_regions_onshore[f"{carrier}_cap"] = capacity_links(n, carrier) / 1000
        df_regions_onshore[f"{carrier}_cf"] = generation_links(n, carrier) / (capacity_links(n, carrier) * 2920)

    # function for carriers in n.storage_units.carrier.unique() # 2 / 2
    for carrier in n.storage_units.carrier.unique():
        df_regions_onshore[f"{carrier}_mv"] = market_values_storage_units(n, carrier)
        df_regions_onshore[f"{carrier}_gen"] = generation_storage_units(n, carrier) / 1000 * 3
        df_regions_onshore[f"{carrier}_cap"] = capacity_storage_units(n, carrier) / 1000
        # capacity factors (both generation and consumption(loading) is considered
        gen = abs(n.storage_units_t.p.loc[:, n.storage_units.carrier == carrier])
        gen.columns = gen.columns.map(n.storage_units.bus)
        gen.columns = gen.columns.map(n.buses.location)
        df_regions_onshore[f"{carrier}_cf"] = gen.sum() / (2*capacity_storage_units(n, carrier) * 2920)

    # set market values to nan where generation in corresponding region is lower than % quantile ( for generators, links, su)
    qt = 0.2
    for carrier in (n.generators.carrier.unique().tolist() +
                    n.links.carrier.unique().tolist() +
                    n.storage_units.carrier.unique().tolist()):
        # onshore
        index = df_regions_onshore[f"{carrier}_gen"] <= np.nanquantile(df_regions_onshore[f"{carrier}_gen"], qt)
        df_regions_onshore[f"{carrier}_mv_qt"] = df_regions_onshore[f"{carrier}_mv"]
        df_regions_onshore[f"{carrier}_mv_qt"][index] = np.nan

        # offshore
        if carrier in ["offwind-dc", "offwind-ac"]:
            index = df_regions_offshore[f"{carrier}_gen"] <= np.nanquantile(df_regions_offshore[f"{carrier}_gen"], qt)
            df_regions_offshore[f"{carrier}_mv_qt"] = df_regions_offshore[f"{carrier}_mv"]
            df_regions_offshore[f"{carrier}_mv_qt"][index] = np.nan

    # calc lmps at the buses (lmps that are only present for EU (e.g. oil) are nan at the moment)
    # TODO: decide if EU lmps are used as lmp for all regions
    for carrier_bus in n.buses.carrier.unique():
        # index would be same names as the bus (not the location as it is in the index of
        # df_regions_onshore so far -> map location to make sure the right lmp is set
        locs = n.buses.location[n.buses[n.buses.carrier == carrier_bus].index]
        lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier == carrier_bus].index].mean()
        df = pd.concat([lmps, locs], axis=1).rename(columns={0: f"{carrier_bus}_lmp"})
        df.set_index("location", inplace=True)
        if df.size == 1:
            if df.index == "EU":
                df = pd.DataFrame(np.repeat(df.values, 181), index=df_regions_onshore.index,
                                  columns=[f"{carrier_bus}_lmp"])
                df_regions_onshore[f"{carrier_bus}_lmp"] = df[f"{carrier_bus}_lmp"]
        else:
            df_regions_onshore[f"{carrier_bus}_lmp"] = df[f"{carrier_bus}_lmp"]

    if n == stst:
        df_stst_ons = df_regions_onshore
        df_stst_off = df_regions_offshore

    if n == exp:
        df_exp_ons = df_regions_onshore
        df_exp_off = df_regions_offshore

In [78]:
# subdivide all carriers: stst.buses.carrier.unique().tolist()

electricity = ['AC', 'battery', 'Li ion', 'low voltage', 'home battery' ]
hydrogen = ['H2', 'H2 liquid']
heat = ['residential rural heat',
 'residential rural water tanks',
 'services rural heat',
 'services rural water tanks',
 'residential urban decentral heat',
 'residential urban decentral water tanks',
 'services urban decentral heat',
 'services urban decentral water tanks',
 'urban central heat',
 'urban central water tanks']
gas = ['gas', 'biogas', 'gas for industry']
oil = ['oil']
biomass = ['solid biomass','solid biomass for industry']
co2 = ['co2','co2 stored']
process_emisisons = ['process emissions']

c_tags = {
    'AC': "el",
    'battery': "el",
    'Li ion': "el",
    'low voltage': "el",
    'home battery': "el",
    'H2': "h2",
    'H2 liquid': "h2",
    'residential rural heat': "heat",
    'residential rural water tanks': "heat",
    'services rural heat': "heat",
    'services rural water tanks': "heat",
    'residential urban decentral heat': "heat",
    'residential urban decentral water tanks': "heat",
    'services urban decentral heat': "heat",
    'services urban decentral water tanks': "heat",
    'urban central heat': "heat",
    'urban central water tanks': "heat",
    'gas': "gas",
    'biogas': "gas",
    'gas for industry': "gas",
    'oil': "oil",
    'solid biomass': "biom",
    'solid biomass for industry': "biom",
    'co2': "co2",
    'co2 stored': "co2",
    'process emissions': "pe"
}

In [79]:
n = stst
c = "onwind"
c_tags["low voltage"]
tag = c_tags[n.generators[n.generators.carrier == c].bus.map(n.buses.carrier).unique()[0]]

In [80]:
c = "PHS"
n.storage_units[n.storage_units.carrier == c].bus.map(n.buses.carrier).unique()

array(['AC'], dtype=object)

In [89]:
# df_stst_ons, df_stst_off, df_exp_ons, df_exp_off
# conventions: consumption is negative, values in GW, GWh
# subdivide gen, mv, cap and cf into electricity, heat, hydrogen, .

buses = ["bus0", "bus1", "bus2", "bus3", "bus4"]
ps = ["p0", "p1", "p2", "p3", "p4"]

for n, dfs in zip([stst, exp], [[df_stst_ons, df_stst_off], [df_exp_ons, df_exp_off]]):

    # generators: can only generate energy, have only one bus, have only one capacity, can have only one carrier
    for c in n.generators.carrier.unique().tolist():
        # get tag
        c_bus = n.generators[n.generators.carrier == c].bus.map(n.buses.carrier).unique()[0]
        c_tag = c_tags[c_bus]
        for df in dfs:
            df[f"{c}_gen_{c_tag}"] = generation(n, c) / 1000 * 3
            df[f"{c}_cap_gen_{c_tag}"] = capacity(n, c) / 1000
            df[f"{c}_mv_gen_{c_tag}"] = market_values(n, c)
            df[f"{c}_cf_gen_{c_tag}"] = generation(n, c) / (capacity(n, c) * 2920)
            # only for onshore
            if len(df) != 100:
                df[f"{c}_mv-lmp_gen_{c_tag}"] = df[f"{c}_mv_gen_{c_tag}"] - df[f"{c_bus}_lmp"]
                df[f"{c}_vf_gen_{c_tag}"] = df[f"{c}_mv_gen_{c_tag}"] / df[f"{c_bus}_lmp"]

    # storage units: can generate and consume, have only one bus, have twoe capacities?, can have only one carrier
    for c in n.storage_units.carrier.unique().tolist():
        # get tag
        c_bus = n.storage_units[n.storage_units.carrier == c].bus.map(n.buses.carrier).unique()[0]
        c_tag = c_tags[c_bus]
        for df in dfs:
            df[f"{c}_gen_{c_tag}"] = generation_storage_units(n, c) / 1000 * 3
            df[f"{c}_con_{c_tag}"] = consumption_storage_units(n, c) / 1000 * 3
            df[f"{c}_cap_gen_{c_tag}"] = capacity_storage_units(n, c) / 1000
            df[f"{c}_cap_con_{c_tag}"] = capacity_storage_units_con(n, c) / 1000
            df[f"{c}_mv_gen_{c_tag}"] = market_values_storage_units(n, c)
            df[f"{c}_mv_con_{c_tag}"] = market_values_storage_units_con(n, c)
            df[f"{c}_cf_gen_{c_tag}"] = generation_storage_units(n, c) / (capacity_storage_units(n, c) * 2920)
            df[f"{c}_cf_con_{c_tag}"] = consumption_storage_units(n, c) / (capacity_storage_units(n, c) * 2920)
            # capacity factor of consumption and generation
            df[f"{c}_cf_gen+con_{c_tag}"] = (generation_storage_units(n, c) + consumption_storage_units(n, c)) / ((capacity_storage_units(n, c) + capacity_storage_units_con(n, c)) * 2920)
            # only for onshore
            if len(df) != 100:
                df[f"{c}_mv-lmp_gen_{c_tag}"] = df[f"{c}_mv_gen_{c_tag}"] - df[f"{c_bus}_lmp"]
                df[f"{c}_vf_gen_{c_tag}"] = df[f"{c}_mv_gen_{c_tag}"] / df[f"{c_bus}_lmp"]
                df[f"{c}_mv-lmp_con_{c_tag}"] = df[f"{c}_mv_con_{c_tag}"] - df[f"{c_bus}_lmp"]
                df[f"{c}_vf_con_{c_tag}"] = df[f"{c}_mv_con_{c_tag}"] / df[f"{c_bus}_lmp"]

    # links: can generate and consume, have several buses with different numbers, have several capacities, can have several carriers even per bus (e.g. DAC bus3 generates heat for urban central and urban decentral)
    for c in n.links.carrier.unique():
        for i, bus in enumerate(buses):
            # check if bus exists
            if n.links[n.links.carrier == c][bus][0] != "":
                # tag
                c_bus = n.links[n.links.carrier == c][bus].map(n.buses.carrier).unique()[0]
                c_tag = c_tags[c_bus]

                for df in dfs:
                    # check if consumption or generation
                    gen = generation_links_bus(n, c, i)
                    gen_tag = "gen" if gen.sum() > 0 else "con"
                    df[f"{c}_{gen_tag}_{c_tag}"] = gen / 1000 * 3
                    df[f"{c}_cap_{gen_tag}_{c_tag}"] = capacity_links_bus(n, c, i) / 1000
                    df[f"{c}_cf_{gen_tag}_{c_tag}"] = abs(gen) / (capacity_links_bus(n, c, i) * 2920)
                    df[f"{c}_mv_{gen_tag}_{c_tag}"] = market_values_links_bus(n, c, i)
                    # only for onshore
                    if len(df) != 100:
                        df[f"{c}_mv-lmp_{gen_tag}_{c_tag}"] = df[f"{c}_mv_{gen_tag}_{c_tag}"] - df[f"{c_bus}_lmp"]
                        df[f"{c}_vf_{gen_tag}_{c_tag}"] = df[f"{c}_mv_{gen_tag}_{c_tag}"] / df[f"{c_bus}_lmp"]

# set market values to nan where generation / consumption in corresponding region is lower / higher than 20%  / 80 % quantile
gen_qt = 0.2
con_qt = 0.8

for df in [df_stst_ons, df_stst_off, df_exp_ons, df_exp_off]:

    # generation
    for col in df.columns[df.columns.str.contains("mv_gen")].tolist():
        index = df[col.replace("mv_", "")] <= np.nanquantile(df[col.replace("mv_", "")], gen_qt)
        df[f"{col}_qt"] = df[col]
        df[f"{col}_qt"][index] = np.nan

    #consumption
    for col in df.columns[df.columns.str.contains("mv_con")].tolist():
        index = df[col.replace("mv_", "")] >= np.nanquantile(df[col.replace("mv_", "")], con_qt)
        df[f"{col}_qt"] = df[col]
        df[f"{col}_qt"][index] = np.nan

    # generation
    for col in df.columns[df.columns.str.contains("vf_gen")].tolist():
        index = df[col.replace("vf_", "")] <= np.nanquantile(df[col.replace("vf_", "")], gen_qt)
        df[f"{col}_qt"] = df[col]
        df[f"{col}_qt"][index] = np.nan

    #consumption
    for col in df.columns[df.columns.str.contains("vf_con")].tolist():
        index = df[col.replace("vf_", "")] >= np.nanquantile(df[col.replace("vf_", "")], con_qt)
        df[f"{col}_qt"] = df[col]
        df[f"{col}_qt"][index] = np.nan

    # generation
    for col in df.columns[df.columns.str.contains("mv-lmp_gen")].tolist():
        index = df[col.replace("mv-lmp_", "")] <= np.nanquantile(df[col.replace("mv-lmp_", "")], gen_qt)
        df[f"{col}_qt"] = df[col]
        df[f"{col}_qt"][index] = np.nan

    #consumption
    for col in df.columns[df.columns.str.contains("mv-lmp_con")].tolist():
        index = df[col.replace("mv-lmp_", "")] >= np.nanquantile(df[col.replace("mv-lmp_", "")], con_qt)
        df[f"{col}_qt"] = df[col]
        df[f"{col}_qt"][index] = np.nan

In [84]:
df_stst_ons["onwind_vf_gen_el_qt"]

name
AL0 0         NaN
AT0 0    0.761653
AT0 1         NaN
AT0 2         NaN
BA0 0         NaN
           ...   
SE3 5         NaN
SE3 6    0.683078
SE3 7    0.961214
SI0 0         NaN
SK0 0    0.823747
Name: onwind_vf_gen_el_qt, Length: 181, dtype: float64

### Consumed electric energy and prices payed

In [83]:
# consumed electric energy and prices payed
# AC and low voltage are the main buses for electricity (why the difference?) Investigate in differences in lmps?

for n , df in zip([stst, exp], [df_stst_ons, df_exp_ons]):

    for c in c_el_con_s:

        # links
        if c in n.links.carrier.unique().tolist():
            # check if bus 0 is AC or low voltage bus
            if n.links[n.links.carrier == c].bus0.map(n.buses.carrier).unique() in ["AC", "low voltage"]:

                # consumption of link at bus 0
                con = n.links_t.p0.loc[:, n.links.carrier == c]
                con.columns = con.columns.map(n.links.bus0)
                # save consumption per location to df
                con_sum = con.sum()
                con_sum.index = con_sum.index.map(n.buses.location)
                # convert to TWh and make negative
                df[f"{c}_con_el2"] = con_sum / 1000 * 3 * -1
                # get lmp of buses where the links consumes from
                lmp_con = n.buses_t.marginal_price.loc[:, con.columns]
                # calculate costs for every time step and location
                overall_cost = con * lmp_con
                # calc consumption weighted average per location: overall cost per location / overall generation per location
                cost_mv = overall_cost.sum() / con.sum()
                cost_mv.index = cost_mv.index.map(n.buses.location)
                # save cost_mv to df (€/MWH_el)
                df[f"{c}_cost_mv_el"] = cost_mv

        # storage unit: only PHS can store
        elif c in n.storage_units.carrier.unique().tolist():

                # consumption of su
                con = n.storage_units_t.p_store.loc[:, n.storage_units.carrier == c]
                con.columns = con.columns.map(n.storage_units.bus)
                # save consumption per location to df in TWh and make negative
                df[f"{c}_con_el2"] = con.sum() / 1000 * 3 * -1
                # get lmp of buses where the su consumes from
                lmp_con = n.buses_t.marginal_price.loc[:, con.columns]
                # calculate costs for every time step and location
                overall_cost = con * lmp_con
                # calc consumption weighted average per location: overall cost per location / overall generation per location
                cost_mv = overall_cost.sum() / con.sum()
                # save cost_mv to df
                df[f"{c}_cost_mv_el"] = cost_mv

        else:
            print(f"{c} not found!")

        # set pries to nan where consumption in corresponding region is lower than % quantile
        # watch out for negative values here!!!!
        qt = 0.8
        index = df[f"{c}_con_el2"] >= np.nanquantile(df[f"{c}_con_el2"], qt)
        df[f"{c}_cost_mv_el_qt"] = df[f"{c}_cost_mv_el"]
        df[f"{c}_cost_mv_el_qt"][index] = np.nan

### Utilisation rate

In [46]:
# utilisation rate

# what you actually want to measure is on how much of possible generation is utilized
# utilitazation rate???
# Problem if you calc the rate for every time step and location independently and then take the mean, all urs have the same weight. That makes no sense, as at times with almost no generation there is numerical issues with the rate
# better calc ur for every region as the ratio of the sum over all gen and the sum over all possible gen (make sure only valid regions make it to the plot, e.g. minimun generation of o,2 quantile)

th = 0.01 # MWh
method = 1

th_p = 0.01 # share of mean generation

# how much of the whole possible energy that can be generated is utilized (sum over all time steps and location than take ratio)
overall_ur_stst = pd.DataFrame(index=range(1))
overall_ur_exp = pd.DataFrame(index=range(1))

for n, df, overall_ur in zip([stst, exp], [df_stst_ons, df_exp_ons], [overall_ur_stst, overall_ur_exp]):

    # generators
    max_out_gen = n.generators_t.p_max_pu * n.generators.p_nom_opt[n.generators_t.p_max_pu.columns]
    real_out_gen = n.generators_t.p[n.generators_t.p_max_pu.columns]
    out_ratio_gen = (real_out_gen / max_out_gen) [real_out_gen > th]
    out_ratio_gen_sum = real_out_gen.sum() / max_out_gen.sum()

    # links
    # gen
    n_links_p1 = n.links_t.p1 *-1
    # some links have a static p_max_pu value and some have an alternating (series)
    index_series_li = n.links_t.p_max_pu.columns
    index_static_li = n.links.index.difference(n.links_t.p_max_pu.columns)

    # calculate the possible output for the link for every time step
    if method == 1:
        # use p_max_pu * p_nom_op
        max_output_links_static = n.links.loc[index_static_li].p_max_pu * n.links.loc[index_static_li].p_nom_opt
    elif method == 2:
        # alternatively use the maximum of the real output and set it as the maximum capacity
        max_output_links_static= n_links_p1[index_static_li].max()

    # make ts of max_output_links_static
    max_output_links_ts_static = n.links_t.p0[index_static_li].copy()
    for snap in n.links_t.p0.index:
        max_output_links_ts_static.loc[snap] = max_output_links_static[index_static_li]

    # calc time series of time dependent p_max_pu links
    max_output_links_ts_series = n.links_t.p_max_pu * n.links.p_nom_opt[index_series_li]

    # merge static and series values and reorder columns
    max_output_links_ts = pd.concat([max_output_links_ts_static, max_output_links_ts_series], axis=1)[n.links_t.p0.columns]

    # compare to real output
    out_ratio_links = (n_links_p1 / max_output_links_ts) [n_links_p1 > th]
    # calc out ratio weighted by generation (sum of all gen / sum of all cap / max_output)
    out_ratio_links_sum = n_links_p1.sum() / max_output_links_ts.sum()

    # storage units
    max_output = n.storage_units.p_max_pu * n.storage_units.p_nom_opt
    max_output_ts_su = n.storage_units_t.p.copy()
    for snap in n.storage_units_t.p.index:
        max_output_ts_su.loc[snap] = max_output[n.storage_units_t.p.columns]
    # compare to real output
    out_ratio_su = (n.storage_units_t.p_dispatch / max_output_ts_su) [n.storage_units_t.p_dispatch > th]
    out_ratio_su_sum = n.storage_units_t.p_dispatch.sum() / max_output_ts_su.sum()

    #######
    #######

    # gens
    for carrier in n.generators.carrier.unique():
        if carrier in ['gas', 'oil']:
            continue
        # calc ur as mean of all urs per time and space
        index = n.generators[n.generators.carrier == carrier].index
        ur = out_ratio_gen[index].mean()
        ur.index = ur.index.map(n.generators.bus).map(n.buses.location)
        df[f"{carrier}_ur_mean"] = ur

        # calc ur as ratio of sum of all gen and sum of all output
        ur_s = out_ratio_gen_sum[index][real_out_gen[index].sum() > real_out_gen[index].sum().mean() * th_p]
        ur_s.index = ur_s.index.map(n.generators.bus).map(n.buses.location)
        df[f"{carrier}_ur"] = ur_s

        # overall ur
        overall_ur[f"{carrier}"] = real_out_gen[index].sum().sum() / max_out_gen[index].sum().sum()

    # links
    for carrier in n.links.carrier.unique():
        # calc
        index = n.links[n.links.carrier == carrier].index
        ur = out_ratio_links[index].mean()
        ur.index = ur.index.map(n.links.bus1).map(n.buses.location)
        # group duplicate index entries
        ur = ur.groupby(by=["Link"], axis="index").mean()
        df[f"{carrier}_ur_mean"] = ur

        # calc ur as ratio of sum of all gen and sum of all output
        ur_s = out_ratio_links_sum[index][n_links_p1[index].sum() > n_links_p1[index].sum().mean() * th_p]
        ur_s.index = ur_s.index.map(n.links.bus1).map(n.buses.location)
        # group duplicate index entries
        ur_s = ur_s.groupby(by=["Link"], axis="index").mean()
        df[f"{carrier}_ur"] = ur_s

        # overall ur
        overall_ur[f"{carrier}"] = n_links_p1[index].sum().sum() / max_output_links_ts[index].sum().sum()

    # storage units
    for carrier in n.storage_units.carrier.unique():
        # calc
        index = n.storage_units[n.storage_units.carrier == carrier].index
        ur = out_ratio_su[index].mean()
        ur.index = ur.index.map(n.storage_units.bus).map(n.buses.location)
        df[f"{carrier}_ur_mean"] = ur

        #
        ur_s = out_ratio_su_sum[index][n.storage_units_t.p_dispatch[index].sum() > n.storage_units_t.p_dispatch[index].sum().mean() * th_p]
        ur_s.index = ur_s.index.map(n.storage_units.bus).map(n.buses.location)
        df[f"{carrier}_ur"] = ur_s

        # overall ur
        overall_ur[f"{carrier}"] = n.storage_units_t.p_dispatch[index].sum().sum() / max_output_ts_su[index].sum().sum()


In [47]:
df_stst_ons.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cap,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cap,...,electricity distribution grid_ur_mean,electricity distribution grid_ur,home battery charger_ur_mean,home battery charger_ur,home battery discharger_ur_mean,home battery discharger_ur,hydro_ur_mean,hydro_ur,PHS_ur_mean,PHS_ur
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((20.32207 39.91318, 20.39703 39.81809...","(20.00648437943388, 41.14563914300007)",AL0 0,126.148417,0.029238,0.000037,0.090705,131.075254,0.015125,0.000045,...,0.563977,0.527220,0.455364,0.454625,0.436440,0.436440,NaN,NaN,NaN,NaN
AT0 0,"POLYGON ((16.45660 47.41184, 16.43396 47.39685...","(15.700718667666258, 47.896990619088925)",AT0 0,NaN,NaN,NaN,NaN,83.457109,30395.904252,23.183878,...,0.734131,0.734131,0.443735,0.443452,0.425714,0.425714,0.555810,0.383551,NaN,NaN
AT0 1,"POLYGON ((10.08288 47.35907, 10.20928 47.37248...","(10.379200993764837, 47.18231496200006)",AT0 1,NaN,NaN,NaN,NaN,84.612302,0.000348,0.000008,...,0.769134,0.769134,0.414851,0.413955,0.398248,0.397396,0.561114,0.383557,0.085168,0.078839
AT0 2,"POLYGON ((13.47802 46.56357, 13.23111 46.55217...","(13.805469952867842, 47.57283376100011)",AT0 2,NaN,NaN,NaN,NaN,90.012931,0.014320,0.000042,...,0.782821,0.782821,0.443474,0.442636,0.424931,0.424931,0.529295,0.383558,0.312799,0.229673
BA0 0,"POLYGON ((17.82716 42.85312, 17.81176 42.90986...","(17.97055162265331, 43.88773569750005)",BA0 0,NaN,NaN,NaN,NaN,119.715551,0.014059,0.000050,...,0.721541,0.702079,0.449194,0.447618,0.429713,0.429713,NaN,NaN,NaN,NaN


In [48]:
df_stst_off.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cap,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cap,...,Fischer-Tropsch_mv_con_h2_qt,Fischer-Tropsch_mv_con_co2_qt,process emissions_mv_con_pe_qt,process emissions CC_mv_con_pe_qt,DAC_mv_con_co2_qt,DAC_mv_con_el_qt,DAC_mv_con_heat_qt,electricity distribution grid_mv_con_el_qt,home battery charger_mv_con_el_qt,home battery discharger_mv_con_el_qt
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((19.98542 39.69475, 20.01003 39.69120...","(19.04227857401803, 40.790848115835146)",AL0 0,126.148417,0.029238,0.000037,0.090705,131.075254,0.015125,0.000045,...,77.368912,NaN,NaN,NaN,NaN,65.527532,25.439011,NaN,NaN,NaN
BE0 0,"POLYGON ((3.25367 51.34858, 3.10403 51.31485, ...","(2.728512798889029, 51.5008335)",BE0 0,78.122119,6139.041623,1.781974,0.393274,73.103671,9140.702915,3.339024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.570056,NaN,NaN
BG0 0,"POLYGON ((29.34583 42.24111, 28.32389 41.99778...","(29.28183834594244, 42.87333189200004)",BG0 0,95.498412,0.148386,0.000096,0.176485,87.582734,11052.570807,8.847056,...,NaN,NaN,NaN,NaN,NaN,68.697408,29.538885,104.376579,77.222146,89.178247
DE0 11,"POLYGON ((8.28604 53.42089, 8.24317 53.41079, ...","(6.336794924479278, 54.505438310008)",DE0 11,53.873868,0.669227,0.000210,0.364074,44.498808,43594.793974,16.821369,...,85.206181,NaN,NaN,NaN,NaN,60.492987,19.160066,72.757860,64.131981,69.611323
DE0 15,"MULTIPOLYGON (((8.81986 54.19861, 8.81277 54.1...","(8.17932998604675, 54.53228697800006)",DE0 15,62.126430,0.790338,0.000282,0.319996,52.030347,89617.262854,31.903299,...,90.450852,NaN,NaN,NaN,NaN,64.050590,24.421675,81.894083,73.653738,80.927485


In [49]:
df_exp_ons.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cap,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cap,...,electricity distribution grid_ur_mean,electricity distribution grid_ur,home battery charger_ur_mean,home battery charger_ur,home battery discharger_ur_mean,home battery discharger_ur,hydro_ur_mean,hydro_ur,PHS_ur_mean,PHS_ur
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((20.32207 39.91318, 20.39703 39.81809...","(20.00648437943388, 41.14563914300007)",AL0 0,106.977597,0.016331,0.000021,0.087899,106.502516,0.016423,0.000043,...,0.637399,0.509638,0.495306,0.493596,0.474148,0.473852,NaN,NaN,NaN,NaN
AT0 0,"POLYGON ((16.45660 47.41184, 16.43396 47.39685...","(15.700718667666258, 47.896990619088925)",AT0 0,NaN,NaN,NaN,NaN,79.253841,0.567355,0.000448,...,0.783561,0.783561,0.477441,0.475798,0.457326,0.456767,0.532048,0.383553,NaN,NaN
AT0 1,"POLYGON ((10.08288 47.35907, 10.20928 47.37248...","(10.379200993764837, 47.18231496200006)",AT0 1,NaN,NaN,NaN,NaN,55.398932,0.000326,0.000008,...,0.762499,0.762072,0.421303,0.419260,0.403554,0.402489,0.504497,0.383557,0.070841,0.067639
AT0 2,"POLYGON ((13.47802 46.56357, 13.23111 46.55217...","(13.805469952867842, 47.57283376100011)",AT0 2,NaN,NaN,NaN,NaN,76.058289,0.012936,0.000033,...,0.792735,0.792735,0.498795,0.497374,0.478063,0.477479,0.474170,0.383558,0.242249,0.197699
BA0 0,"POLYGON ((17.82716 42.85312, 17.81176 42.90986...","(17.97055162265331, 43.88773569750005)",BA0 0,NaN,NaN,NaN,NaN,92.921744,0.010620,0.000038,...,0.724753,0.709851,0.507094,0.505647,0.485725,0.485421,NaN,NaN,NaN,NaN


In [50]:
df_exp_off.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cap,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cap,...,Fischer-Tropsch_mv_con_h2_qt,Fischer-Tropsch_mv_con_co2_qt,process emissions_mv_con_pe_qt,process emissions CC_mv_con_pe_qt,DAC_mv_con_co2_qt,DAC_mv_con_el_qt,DAC_mv_con_heat_qt,electricity distribution grid_mv_con_el_qt,home battery charger_mv_con_el_qt,home battery discharger_mv_con_el_qt
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((19.98542 39.69475, 20.01003 39.69120...","(19.04227857401803, 40.790848115835146)",AL0 0,106.977597,0.016331,0.000021,0.087899,106.502516,0.016423,0.000043,...,84.766504,NaN,NaN,NaN,NaN,63.016600,29.635572,NaN,70.189710,75.532580
BE0 0,"POLYGON ((3.25367 51.34858, 3.10403 51.31485, ...","(2.728512798889029, 51.5008335)",BE0 0,56.352353,6343.446483,1.781972,0.406369,52.212890,9585.550789,3.339025,...,85.820144,NaN,NaN,NaN,NaN,59.826068,30.698091,76.869220,NaN,NaN
BG0 0,"POLYGON ((29.34583 42.24111, 28.32389 41.99778...","(29.28183834594244, 42.87333189200004)",BG0 0,94.568790,0.100601,0.000064,0.178239,85.769797,8.556417,0.006758,...,84.234526,NaN,NaN,NaN,NaN,63.840659,31.105908,85.731845,70.062562,75.908383
DE0 11,"POLYGON ((8.28604 53.42089, 8.24317 53.41079, ...","(6.336794924479278, 54.505438310008)",DE0 11,51.455031,8441.804726,2.216633,0.434748,47.257302,45018.091917,16.821408,...,85.775631,NaN,NaN,NaN,NaN,56.850463,31.887282,71.917315,64.423023,66.613677
DE0 15,"MULTIPOLYGON (((8.81986 54.19861, 8.81277 54.1...","(8.17932998604675, 54.53228697800006)",DE0 15,60.568652,0.306136,0.000113,0.308317,50.245283,89980.590188,31.903305,...,85.894441,NaN,NaN,NaN,NaN,58.959474,31.528490,74.722476,NaN,NaN


In [90]:
# save to pickle
df_stst_ons.to_pickle("../data/processed/df_stst_ons.pkl")
df_stst_off.to_pickle("../data/processed/df_stst_off.pkl")
df_exp_ons.to_pickle("../data/processed/df_exp_ons.pkl")
df_exp_off.to_pickle("../data/processed/df_exp_off.pkl")

overall_ur_stst.to_pickle("../data/processed/overall_ur_stst.pkl")
overall_ur_exp.to_pickle("../data/processed/overall_ur_exp.pkl")

In [91]:
# verify reload
df_stst_ons = pd.read_pickle("../data/processed/df_stst_ons.pkl")
df_stst_off = pd.read_pickle("../data/processed/df_stst_off.pkl")
df_exp_ons = pd.read_pickle("../data/processed/df_exp_ons.pkl")
df_exp_off = pd.read_pickle("../data/processed/df_exp_off.pkl")

In [53]:
df_stst_ons.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cap,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cap,...,electricity distribution grid_ur_mean,electricity distribution grid_ur,home battery charger_ur_mean,home battery charger_ur,home battery discharger_ur_mean,home battery discharger_ur,hydro_ur_mean,hydro_ur,PHS_ur_mean,PHS_ur
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((20.32207 39.91318, 20.39703 39.81809...","(20.00648437943388, 41.14563914300007)",AL0 0,126.148417,0.029238,0.000037,0.090705,131.075254,0.015125,0.000045,...,0.563977,0.527220,0.455364,0.454625,0.436440,0.436440,NaN,NaN,NaN,NaN
AT0 0,"POLYGON ((16.45660 47.41184, 16.43396 47.39685...","(15.700718667666258, 47.896990619088925)",AT0 0,NaN,NaN,NaN,NaN,83.457109,30395.904252,23.183878,...,0.734131,0.734131,0.443735,0.443452,0.425714,0.425714,0.555810,0.383551,NaN,NaN
AT0 1,"POLYGON ((10.08288 47.35907, 10.20928 47.37248...","(10.379200993764837, 47.18231496200006)",AT0 1,NaN,NaN,NaN,NaN,84.612302,0.000348,0.000008,...,0.769134,0.769134,0.414851,0.413955,0.398248,0.397396,0.561114,0.383557,0.085168,0.078839
AT0 2,"POLYGON ((13.47802 46.56357, 13.23111 46.55217...","(13.805469952867842, 47.57283376100011)",AT0 2,NaN,NaN,NaN,NaN,90.012931,0.014320,0.000042,...,0.782821,0.782821,0.443474,0.442636,0.424931,0.424931,0.529295,0.383558,0.312799,0.229673
BA0 0,"POLYGON ((17.82716 42.85312, 17.81176 42.90986...","(17.97055162265331, 43.88773569750005)",BA0 0,NaN,NaN,NaN,NaN,119.715551,0.014059,0.000050,...,0.721541,0.702079,0.449194,0.447618,0.429713,0.429713,NaN,NaN,NaN,NaN


In [54]:
df_stst_ons.head().dtypes

geometry                      geometry
coords                          object
name                            object
offwind-ac_mv                  float64
offwind-ac_gen                 float64
                                ...   
home battery discharger_ur     float64
hydro_ur_mean                  float64
hydro_ur                       float64
PHS_ur_mean                    float64
PHS_ur                         float64
Length: 1271, dtype: object

In [55]:
df_stst_off.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cap,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cap,...,Fischer-Tropsch_mv_con_h2_qt,Fischer-Tropsch_mv_con_co2_qt,process emissions_mv_con_pe_qt,process emissions CC_mv_con_pe_qt,DAC_mv_con_co2_qt,DAC_mv_con_el_qt,DAC_mv_con_heat_qt,electricity distribution grid_mv_con_el_qt,home battery charger_mv_con_el_qt,home battery discharger_mv_con_el_qt
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((19.98542 39.69475, 20.01003 39.69120...","(19.04227857401803, 40.790848115835146)",AL0 0,126.148417,0.029238,0.000037,0.090705,131.075254,0.015125,0.000045,...,77.368912,NaN,NaN,NaN,NaN,65.527532,25.439011,NaN,NaN,NaN
BE0 0,"POLYGON ((3.25367 51.34858, 3.10403 51.31485, ...","(2.728512798889029, 51.5008335)",BE0 0,78.122119,6139.041623,1.781974,0.393274,73.103671,9140.702915,3.339024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.570056,NaN,NaN
BG0 0,"POLYGON ((29.34583 42.24111, 28.32389 41.99778...","(29.28183834594244, 42.87333189200004)",BG0 0,95.498412,0.148386,0.000096,0.176485,87.582734,11052.570807,8.847056,...,NaN,NaN,NaN,NaN,NaN,68.697408,29.538885,104.376579,77.222146,89.178247
DE0 11,"POLYGON ((8.28604 53.42089, 8.24317 53.41079, ...","(6.336794924479278, 54.505438310008)",DE0 11,53.873868,0.669227,0.000210,0.364074,44.498808,43594.793974,16.821369,...,85.206181,NaN,NaN,NaN,NaN,60.492987,19.160066,72.757860,64.131981,69.611323
DE0 15,"MULTIPOLYGON (((8.81986 54.19861, 8.81277 54.1...","(8.17932998604675, 54.53228697800006)",DE0 15,62.126430,0.790338,0.000282,0.319996,52.030347,89617.262854,31.903299,...,90.450852,NaN,NaN,NaN,NaN,64.050590,24.421675,81.894083,73.653738,80.927485


In [56]:
df_exp_ons.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cap,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cap,...,electricity distribution grid_ur_mean,electricity distribution grid_ur,home battery charger_ur_mean,home battery charger_ur,home battery discharger_ur_mean,home battery discharger_ur,hydro_ur_mean,hydro_ur,PHS_ur_mean,PHS_ur
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((20.32207 39.91318, 20.39703 39.81809...","(20.00648437943388, 41.14563914300007)",AL0 0,106.977597,0.016331,0.000021,0.087899,106.502516,0.016423,0.000043,...,0.637399,0.509638,0.495306,0.493596,0.474148,0.473852,NaN,NaN,NaN,NaN
AT0 0,"POLYGON ((16.45660 47.41184, 16.43396 47.39685...","(15.700718667666258, 47.896990619088925)",AT0 0,NaN,NaN,NaN,NaN,79.253841,0.567355,0.000448,...,0.783561,0.783561,0.477441,0.475798,0.457326,0.456767,0.532048,0.383553,NaN,NaN
AT0 1,"POLYGON ((10.08288 47.35907, 10.20928 47.37248...","(10.379200993764837, 47.18231496200006)",AT0 1,NaN,NaN,NaN,NaN,55.398932,0.000326,0.000008,...,0.762499,0.762072,0.421303,0.419260,0.403554,0.402489,0.504497,0.383557,0.070841,0.067639
AT0 2,"POLYGON ((13.47802 46.56357, 13.23111 46.55217...","(13.805469952867842, 47.57283376100011)",AT0 2,NaN,NaN,NaN,NaN,76.058289,0.012936,0.000033,...,0.792735,0.792735,0.498795,0.497374,0.478063,0.477479,0.474170,0.383558,0.242249,0.197699
BA0 0,"POLYGON ((17.82716 42.85312, 17.81176 42.90986...","(17.97055162265331, 43.88773569750005)",BA0 0,NaN,NaN,NaN,NaN,92.921744,0.010620,0.000038,...,0.724753,0.709851,0.507094,0.505647,0.485725,0.485421,NaN,NaN,NaN,NaN


In [57]:
df_exp_off.head()

,geometry,coords,name,offwind-ac_mv,offwind-ac_gen,offwind-ac_cap,offwind-ac_cf,onwind_mv,onwind_gen,onwind_cap,...,Fischer-Tropsch_mv_con_h2_qt,Fischer-Tropsch_mv_con_co2_qt,process emissions_mv_con_pe_qt,process emissions CC_mv_con_pe_qt,DAC_mv_con_co2_qt,DAC_mv_con_el_qt,DAC_mv_con_heat_qt,electricity distribution grid_mv_con_el_qt,home battery charger_mv_con_el_qt,home battery discharger_mv_con_el_qt
name,,,,,,,,,,,,,,,,,,,,,
AL0 0,"POLYGON ((19.98542 39.69475, 20.01003 39.69120...","(19.04227857401803, 40.790848115835146)",AL0 0,106.977597,0.016331,0.000021,0.087899,106.502516,0.016423,0.000043,...,84.766504,NaN,NaN,NaN,NaN,63.016600,29.635572,NaN,70.189710,75.532580
BE0 0,"POLYGON ((3.25367 51.34858, 3.10403 51.31485, ...","(2.728512798889029, 51.5008335)",BE0 0,56.352353,6343.446483,1.781972,0.406369,52.212890,9585.550789,3.339025,...,85.820144,NaN,NaN,NaN,NaN,59.826068,30.698091,76.869220,NaN,NaN
BG0 0,"POLYGON ((29.34583 42.24111, 28.32389 41.99778...","(29.28183834594244, 42.87333189200004)",BG0 0,94.568790,0.100601,0.000064,0.178239,85.769797,8.556417,0.006758,...,84.234526,NaN,NaN,NaN,NaN,63.840659,31.105908,85.731845,70.062562,75.908383
DE0 11,"POLYGON ((8.28604 53.42089, 8.24317 53.41079, ...","(6.336794924479278, 54.505438310008)",DE0 11,51.455031,8441.804726,2.216633,0.434748,47.257302,45018.091917,16.821408,...,85.775631,NaN,NaN,NaN,NaN,56.850463,31.887282,71.917315,64.423023,66.613677
DE0 15,"MULTIPOLYGON (((8.81986 54.19861, 8.81277 54.1...","(8.17932998604675, 54.53228697800006)",DE0 15,60.568652,0.306136,0.000113,0.308317,50.245283,89980.590188,31.903305,...,85.894441,NaN,NaN,NaN,NaN,58.959474,31.528490,74.722476,NaN,NaN


## Temporal dfs

In [58]:
# calc electricity generation or consumption for every time step

df_stst_ts = pd.DataFrame(index=stst.generators_t.p.index)
df_exp_ts = pd.DataFrame(index=exp.generators_t.p.index)

for n, df in zip([stst, exp],[df_stst_ts, df_exp_ts]):
    for c in (c_el_gen_s + c_el_con_s):

        if c in n.generators.carrier.unique():
            df[f"{c}_gen"] = n.generators_t.p.loc[:, n.generators.carrier == c].sum(axis=1)

        elif c in n.links.carrier.unique():
            df[f"{c}_gen"] = -n.links_t.p1.loc[:, n.links.carrier == c].sum(axis=1)
            df[f"{c}_con_el"] = -n.links_t.p1.loc[:, n.links.carrier == c].sum(axis=1) * -1

        elif c in n.storage_units.carrier.unique():
            df[f"{c}_gen"] = n.storage_units_t.p_dispatch.loc[:, n.storage_units.carrier == c].sum(axis=1)
            df[f"{c}_con_el"] = n.storage_units_t.p_store.loc[:, n.storage_units.carrier == c].sum(axis=1) * -1

        else:
            print(f"{c} is not a known carrier")

In [59]:
df_stst_ts.head()

,onwind_gen,solar_gen,offwind-dc_gen,hydro_gen,hydro_con_el,solar rooftop_gen,battery discharger_gen,battery discharger_con_el,offwind-ac_gen,ror_gen,...,H2 Electrolysis_gen,H2 Electrolysis_con_el,BEV charger_gen,BEV charger_con_el,battery charger_gen,battery charger_con_el,urban central air heat pump_gen,urban central air heat pump_con_el,urban central resistive heater_gen,urban central resistive heater_con_el
snapshot,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00,1.115046e+06,0.000000,96780.482471,32616.407352,-0.0,0.000000,41299.585753,-41299.585753,36717.442882,12320.808629,...,472983.016887,-472983.016887,141501.650312,-141501.650312,9003.183588,-9003.183588,121154.555872,-121154.555872,16023.457529,-16023.457529
2013-01-01 03:00:00,1.055760e+06,0.000000,87476.404567,34583.787290,-0.0,0.000000,50472.110616,-50472.110616,30103.183830,12352.313154,...,431130.292709,-431130.292709,101838.924102,-101838.924102,10103.946175,-10103.946175,122281.212604,-122281.212604,45365.287424,-45365.287424
2013-01-01 06:00:00,9.886784e+05,158083.842075,99085.284393,35854.176137,-0.0,17369.631860,13962.508203,-13962.508203,30304.778869,12489.446573,...,418300.065034,-418300.065034,137797.551009,-137797.551009,33412.155433,-33412.155433,122899.974911,-122899.974911,55752.405009,-55752.405009
2013-01-01 09:00:00,8.329774e+05,831849.855205,106101.418311,24616.469476,-0.0,77017.545359,271.323747,-271.323747,27248.506958,12641.889286,...,616780.479643,-616780.479643,248275.864754,-248275.864754,153979.285016,-153979.285016,126537.038034,-126537.038034,49023.531577,-49023.531577
2013-01-01 12:00:00,8.327530e+05,652761.327098,111841.816426,42924.819243,-0.0,57996.306341,1935.111795,-1935.111795,27101.491865,12971.781726,...,548648.824131,-548648.824131,221648.035099,-221648.035099,141436.903434,-141436.903434,129377.063056,-129377.063056,38437.555919,-38437.555919


In [60]:
df_exp_ts.head()

,onwind_gen,solar_gen,offwind-dc_gen,hydro_gen,hydro_con_el,solar rooftop_gen,battery discharger_gen,battery discharger_con_el,offwind-ac_gen,ror_gen,...,H2 Electrolysis_gen,H2 Electrolysis_con_el,BEV charger_gen,BEV charger_con_el,battery charger_gen,battery charger_con_el,urban central air heat pump_gen,urban central air heat pump_con_el,urban central resistive heater_gen,urban central resistive heater_con_el
snapshot,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00,1.004871e+06,0.000000,132715.807997,16254.378260,-0.0,0.000000,16378.782579,-16378.782579,26814.308286,12359.133966,...,383673.410285,-383673.410285,130743.122562,-130743.122562,22.472782,-22.472782,119774.414877,-119774.414877,25279.984642,-25279.984642
2013-01-01 03:00:00,9.464829e+05,0.000000,119299.611895,16316.852435,-0.0,0.000000,38152.768816,-38152.768816,20431.056746,12390.526455,...,344086.285332,-344086.285332,71910.298594,-71910.298594,19.624660,-19.624660,118784.842481,-118784.842481,65392.837163,-65392.837163
2013-01-01 06:00:00,8.902786e+05,123481.585163,141140.696947,406.617021,-0.0,15266.921462,724.352564,-724.352564,20830.104695,12524.626379,...,306165.039068,-306165.039068,117902.017322,-117902.017322,6551.860796,-6551.860796,118205.728930,-118205.728930,79231.035858,-79231.035858
2013-01-01 09:00:00,7.909976e+05,605243.173189,157478.679277,404.648028,-0.0,65890.035274,12.581618,-12.581618,19272.243401,12710.149548,...,491742.370063,-491742.370063,224795.950456,-224795.950456,54945.071967,-54945.071967,122076.629832,-122076.629832,65018.567538,-65018.567538
2013-01-01 12:00:00,7.799514e+05,490742.596278,173255.846806,423.282346,-0.0,50699.632594,16.483854,-16.483854,19832.735058,12973.790312,...,441456.912589,-441456.912589,203407.228199,-203407.228199,54172.258425,-54172.258425,125193.019625,-125193.019625,56782.152060,-56782.152060


In [61]:
# save to pickle
df_stst_ts.to_pickle("../data/processed/df_stst_ts.pkl")
df_exp_ts.to_pickle("../data/processed/df_exp_ts.pkl")

# verify reload
df_stst_ts = pd.read_pickle("../data/processed/df_stst_ts.pkl")
df_exp_ts = pd.read_pickle("../data/processed/df_exp_ts.pkl")

In [62]:
df_stst_ts.head()

,onwind_gen,solar_gen,offwind-dc_gen,hydro_gen,hydro_con_el,solar rooftop_gen,battery discharger_gen,battery discharger_con_el,offwind-ac_gen,ror_gen,...,H2 Electrolysis_gen,H2 Electrolysis_con_el,BEV charger_gen,BEV charger_con_el,battery charger_gen,battery charger_con_el,urban central air heat pump_gen,urban central air heat pump_con_el,urban central resistive heater_gen,urban central resistive heater_con_el
snapshot,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00,1.115046e+06,0.000000,96780.482471,32616.407352,-0.0,0.000000,41299.585753,-41299.585753,36717.442882,12320.808629,...,472983.016887,-472983.016887,141501.650312,-141501.650312,9003.183588,-9003.183588,121154.555872,-121154.555872,16023.457529,-16023.457529
2013-01-01 03:00:00,1.055760e+06,0.000000,87476.404567,34583.787290,-0.0,0.000000,50472.110616,-50472.110616,30103.183830,12352.313154,...,431130.292709,-431130.292709,101838.924102,-101838.924102,10103.946175,-10103.946175,122281.212604,-122281.212604,45365.287424,-45365.287424
2013-01-01 06:00:00,9.886784e+05,158083.842075,99085.284393,35854.176137,-0.0,17369.631860,13962.508203,-13962.508203,30304.778869,12489.446573,...,418300.065034,-418300.065034,137797.551009,-137797.551009,33412.155433,-33412.155433,122899.974911,-122899.974911,55752.405009,-55752.405009
2013-01-01 09:00:00,8.329774e+05,831849.855205,106101.418311,24616.469476,-0.0,77017.545359,271.323747,-271.323747,27248.506958,12641.889286,...,616780.479643,-616780.479643,248275.864754,-248275.864754,153979.285016,-153979.285016,126537.038034,-126537.038034,49023.531577,-49023.531577
2013-01-01 12:00:00,8.327530e+05,652761.327098,111841.816426,42924.819243,-0.0,57996.306341,1935.111795,-1935.111795,27101.491865,12971.781726,...,548648.824131,-548648.824131,221648.035099,-221648.035099,141436.903434,-141436.903434,129377.063056,-129377.063056,38437.555919,-38437.555919


In [63]:
df_exp_ts.head()

,onwind_gen,solar_gen,offwind-dc_gen,hydro_gen,hydro_con_el,solar rooftop_gen,battery discharger_gen,battery discharger_con_el,offwind-ac_gen,ror_gen,...,H2 Electrolysis_gen,H2 Electrolysis_con_el,BEV charger_gen,BEV charger_con_el,battery charger_gen,battery charger_con_el,urban central air heat pump_gen,urban central air heat pump_con_el,urban central resistive heater_gen,urban central resistive heater_con_el
snapshot,,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00,1.004871e+06,0.000000,132715.807997,16254.378260,-0.0,0.000000,16378.782579,-16378.782579,26814.308286,12359.133966,...,383673.410285,-383673.410285,130743.122562,-130743.122562,22.472782,-22.472782,119774.414877,-119774.414877,25279.984642,-25279.984642
2013-01-01 03:00:00,9.464829e+05,0.000000,119299.611895,16316.852435,-0.0,0.000000,38152.768816,-38152.768816,20431.056746,12390.526455,...,344086.285332,-344086.285332,71910.298594,-71910.298594,19.624660,-19.624660,118784.842481,-118784.842481,65392.837163,-65392.837163
2013-01-01 06:00:00,8.902786e+05,123481.585163,141140.696947,406.617021,-0.0,15266.921462,724.352564,-724.352564,20830.104695,12524.626379,...,306165.039068,-306165.039068,117902.017322,-117902.017322,6551.860796,-6551.860796,118205.728930,-118205.728930,79231.035858,-79231.035858
2013-01-01 09:00:00,7.909976e+05,605243.173189,157478.679277,404.648028,-0.0,65890.035274,12.581618,-12.581618,19272.243401,12710.149548,...,491742.370063,-491742.370063,224795.950456,-224795.950456,54945.071967,-54945.071967,122076.629832,-122076.629832,65018.567538,-65018.567538
2013-01-01 12:00:00,7.799514e+05,490742.596278,173255.846806,423.282346,-0.0,50699.632594,16.483854,-16.483854,19832.735058,12973.790312,...,441456.912589,-441456.912589,203407.228199,-203407.228199,54172.258425,-54172.258425,125193.019625,-125193.019625,56782.152060,-56782.152060
